In [1]:
import numpy as np
import random
from collections import Counter

In [2]:
# Directions on 2d Lattice
DIRECTIONS = [(1, 0), (0, 1), (-1, 0), (0, -1)]

def get_neighbors(pos):
    return [(pos[0] + dx, pos[1] + dy) for dx, dy in DIRECTIONS]

def valid_moves(walk):
    last = walk[-1]
    visited = set(walk)
    return [nbr for nbr in get_neighbors(last) if nbr not in visited]

In [3]:
def initialize_walks(N):
    # Each walk starts at the origin (0, 0) and makes one ranomd step
    walks = []
    for _ in range(N):
        start = (0, 0)
        step = random.choice(DIRECTIONS)
        walks.append([start, (start[0] + step[0], start[1] + step[1])])
    return walks

def extend_walks(walks):
    new_walks = []
    weights = []
    for walk in walks:
        moves = valid_moves(walk=walk)
        k = len(moves)
        if k == 0:
            weights.append(0)
            new_walks.append(walk)
        else:
            step = random.choice(moves)
            new_walk = walk + [step]
            weights.append(k)
            new_walks.append(new_walk)
    return new_walks, weights


In [4]:
def resample(walks, weights):
    total_weight = sum(weights)
    if total_weight == 0:
        raise ValueError("Walks error: all weights are zero")
    probabilities = [weight / total_weight for weight in weights]
    indices = np.random.choice(len(walks), size=len(walks), p=probabilities)
    resampled_walks = [walks[i] for i in indices]
    return resampled_walks

def smc_saw(N=1000, max_len = 20):
    walks = initialize_walks(N=N)
    logZ = 0 # log partition function, for estimating mu

    for L in range(2, max_len+ 1):
        walks, weights = extend_walks(walks=walks)
        avg_weight = np.mean(weights)
        logZ += np.log(avg_weight)
        walks = resample(walks=walks, weights=weights)
        print(f"Length {L}: avg weight = {avg_weight:.4f}")

    mu_est = np.exp(logZ / (max_len -1))
    print(f"Estimated mu: {mu_est:.4f}")
    return mu_est

In [5]:
smc_saw(N=1000, max_len=1000)

Length 2: avg weight = 3.0000
Length 3: avg weight = 3.0000
Length 4: avg weight = 2.7680
Length 5: avg weight = 2.8670
Length 6: avg weight = 2.7750
Length 7: avg weight = 2.7760
Length 8: avg weight = 2.7230
Length 9: avg weight = 2.7350
Length 10: avg weight = 2.7060
Length 11: avg weight = 2.7400
Length 12: avg weight = 2.7260
Length 13: avg weight = 2.7310
Length 14: avg weight = 2.6970
Length 15: avg weight = 2.6730
Length 16: avg weight = 2.6990
Length 17: avg weight = 2.7400
Length 18: avg weight = 2.6800
Length 19: avg weight = 2.6800
Length 20: avg weight = 2.6740
Length 21: avg weight = 2.7020
Length 22: avg weight = 2.6950
Length 23: avg weight = 2.6730
Length 24: avg weight = 2.7030
Length 25: avg weight = 2.6750
Length 26: avg weight = 2.6680
Length 27: avg weight = 2.6510
Length 28: avg weight = 2.7140
Length 29: avg weight = 2.6770
Length 30: avg weight = 2.6760
Length 31: avg weight = 2.5920
Length 32: avg weight = 2.6860
Length 33: avg weight = 2.6770
Length 34: avg w

np.float64(2.64326472451579)